# Movielens 영화 추천 

1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 옮김
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) 작업디렉토리로 이동
$ cd ~/aiffel/recommendata_iu/data

4) 압축 해제
$ unzip ml-1m.zip

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# timestamp 항목은 관심없다.
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id','rating']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies_using_cols = ['movie_id','title']


movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies = movies[movies_using_cols]
movies.head(10)

,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


// 병합 과정이 없으니까 아래 코드들에서 movies에 title을 찾으려 할때 오류가 났다. 병합 후 ratings_add에서 title을 찾으니 해결!

In [7]:
# 영화제목 추가해서 변경해주기 
ratings_add = pd.merge(ratings,movies, on = "movie_id")   # 표 합치기 - movie_id라는 열만 겹친다!
ratings_add

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975)
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975)
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975)
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975)
...,...,...,...,...
836473,5851,3607,5,One Little Indian (1973)
836474,5854,3026,4,Slaughterhouse (1987)
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)"


#### 분석해보기

In [8]:
# ratings에 있는 유니크한 영화 개수
print("영화 갯수 :",ratings_add['movie_id'].nunique())

#rating에 있는 유니크한 사용자 수
print("사용자 수 :",ratings_add['user_id'].nunique())

영화 갯수 : 3628
사용자 수 : 6039


In [9]:
# 가장 인기 있는 영화 30개(인기순)
popular_movie_count = ratings_add.groupby('title')['user_id'].count()
popular_movie_count.sort_values(ascending=False).head(50)


title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 모델 검증을 위한 사용자 초기 정보 세팅
#### 내가 선호하는 영화를 5가지 골라서 rating에 추가하기
- (문제)새로 추가한 영화의 title은 있지만 movie_id 가 없다!!  -- user_id가 인식되지 않음 (데이터셋에서 찾아서 넣어주자...)
- 임의의 movie_id를 넣어주면 user_id 인식 -- 임의의 movie_id 대신 데이터셋 movie_id 찾아주기

In [10]:
# 선호하는 영화 5가지 추가! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ["'One Flew Over the Cuckoo's Nest (1975)" , 'Jumanji (1995)' ,'One Little Indian (1973)' ,'Toy Story (1995)' ,'To Die For (1995)']

my_favorite_id = [1193,2,3607,4,45]  # movie_id 찾아서 입력해주기


# 'mee'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_movie = pd.DataFrame({'user_id': ['mee']*5, 'movie_id' :my_favorite_id ,  'title' : my_favorite,'count':[5]*5})

if not ratings_add.isin({'user_id':['mee']})['user_id'].any():  # user_id에 'mee'이라는 데이터가 없다면
    ratings_add = ratings_add.append(my_movie)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings_add.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
836473,5851,3607,5,One Little Indian (1973)
836474,5854,3026,4,Slaughterhouse (1987)
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)"
836477,5948,1360,5,Identification of a Woman (Identificazione di ...
0,mee,1193,5,'One Flew Over the Cuckoo's Nest (1975)
1,mee,2,5,Jumanji (1995)
2,mee,3607,5,One Little Indian (1973)
3,mee,4,5,Toy Story (1995)
4,mee,45,5,To Die For (1995)


### 모델에 활용하기 위한 전처리 - indexing

In [11]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings_add['user_id'].unique()
movie_unique = ratings_add['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [12]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['mee'])    # 아직 CSR matrix 적용 전이라 6039 ?
print(movie_to_idx['Richard III (1995)'])

6039
218


In [13]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings_add['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings_add):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings_add['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings_add['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings_add):
    print('movie column indexing OK!!')
    ratings_add['title'] = temp_movie_data
    
    
else:
    print('movie column indexing Fail!!')

print("ratings의 user_id와 movie 모두 정수 인덱스 값으로 변경!")    
ratings_add

user_id column indexing OK!!
movie column indexing OK!!
ratings의 user_id와 movie 모두 정수 인덱스 값으로 변경!


,user_id,movie_id,count,title
0,0,1193,5,0
1,1,1193,5,0
2,2,1193,4,0
3,3,1193,4,0
4,4,1193,5,0
...,...,...,...,...
0,6039,1193,5,3628
1,6039,2,5,513
2,6039,3607,5,3623
3,6039,4,5,40


#### CSR matrix 만들기

In [14]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings_add['user_id'].nunique()   # 6040으로 늘어남!
num_movie = ratings_add['title'].nunique()

num_user, num_movie

(6040, 3629)

In [15]:
csr_ratings = csr_matrix((ratings_add['count'], (ratings_add.user_id, ratings_add.title))) # 인덱스 차원 오류 나타나서 빼줌 - shape= (num_user, num_movie)
csr_ratings

<6040x3629 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

#### als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시키기

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_ratings_transpose = csr_ratings.T
csr_ratings_transpose

<3629x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_ratings_transpose)
print("모델훈련 완료!")

  0%|          | 0/15 [00:00<?, ?it/s]

모델훈련 완료!


#### 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악하기
- 1) 모델이 '나'의 벡터와 'Aladin'벡터를 어떻게 만들고 있는지 
- 2) 두 벡터를 곱하면 어떤 값이 나오는지 

In [34]:
mee, Toy_story = user_to_idx['mee'], movie_to_idx['Toy Story (1995)']
mee_vector, Toy_story_vector = als_model.user_factors[mee], als_model.item_factors[Toy_story]

print('모델을 통해 구할 벡터 구성 완료')

모델을 통해 구할 벡터 구성 완료


In [31]:
mee_vector

array([-0.0394251 ,  0.25375006,  0.05379024, -0.23094298, -0.6668837 ,
       -0.37436533, -0.07260381,  0.14178623, -0.06973664,  0.14333645,
        0.03616124, -0.9209283 , -0.04997108,  0.64519703,  0.19769193,
       -0.49853867,  0.00291126, -0.31804162,  0.2502424 ,  0.41769403,
        0.02873964, -0.12665167, -0.7278591 , -0.37857205, -0.71688414,
        0.11428843,  0.4411494 , -0.02807147,  0.11636631,  0.47711396,
       -0.5438468 , -0.15239541, -0.16205183, -0.1924766 ,  0.325616  ,
        0.25804976,  0.05761448,  0.15549351, -0.48398077,  0.27316675,
       -0.67001766,  0.06019577, -0.39150846, -0.5395485 ,  0.43545127,
       -0.14252244, -0.15358868, -0.23391384,  0.13548963,  0.24400973,
        0.15305516,  0.40698305,  0.3118881 ,  0.21373403, -0.07098311,
        0.33029532, -0.12686744,  0.48598528, -0.00709015,  0.20500039,
        0.1611849 , -0.32959923,  0.01593384, -0.17030399,  0.02168088,
        0.6446338 , -0.33237067, -0.18983606,  0.521776  ,  0.47

In [35]:
Toy_story_vector

array([ 0.00328621, -0.00241506, -0.01809818, -0.00426363, -0.0165383 ,
       -0.02314479,  0.01128219,  0.03524018, -0.00114158,  0.02088149,
        0.00253746,  0.00401529, -0.00653932,  0.02226815,  0.04266822,
       -0.01910793, -0.00052527,  0.00406431,  0.0175462 ,  0.01304028,
        0.00910094, -0.02938518, -0.00356675, -0.01074811, -0.01210991,
        0.0277902 , -0.00886005, -0.01040279, -0.01459656,  0.01513283,
       -0.03789714, -0.00352111,  0.00871178,  0.01824656,  0.01310392,
        0.04115353,  0.03667932,  0.01109257, -0.02284145, -0.01169943,
       -0.01262885,  0.01584234, -0.00598863, -0.02558811,  0.01425833,
        0.03576908,  0.01692779, -0.00481525,  0.02181703,  0.01619314,
        0.0365669 ,  0.03441244,  0.00894968,  0.02381778, -0.0007432 ,
        0.00543478,  0.00273582,  0.03531898, -0.00785589,  0.00577765,
        0.01400461, -0.0153745 ,  0.02776775,  0.00948268,  0.02187466,
        0.03439125,  0.00317473, -0.01484832,  0.01629927,  0.04

In [36]:
# mee 와 Toy_story 내적하는 코드
np.dot(mee_vector,Toy_story_vector)

0.4292755

#### 그 외 영화 하나 훈련 - Toy Story (1995)	

In [45]:
Other_movie = movie_to_idx['Sudden Death (1995)']
Other_movie_vector = als_model.item_factors[Other_movie]

print('모델을 통해 구할 벡터 구성 완료2')

모델을 통해 구할 벡터 구성 완료2


In [46]:
# mee 와 Other_movie 내적하는 코드
np.dot(mee_vector,Other_movie_vector)

0.0034316303

#### 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
- AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드 이용

In [66]:
# favorite_movie = 'Toy Story (1995)'
# movie_id = movie_to_idx[favorite_movie]
# similar_movie = als_model.similar_items(movie_id, N=15)   #N=15 의미?
# similar_movie

In [65]:
# # (movie의 id, 유사도) Tuple 로 반환하고 있습니다. 
# # movie의 id를 다시 movie의 이름으로 매핑 시켜 주겠습니다.

# #movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
# idx_to_movie = {v:k for k,v in movie_to_idx.items()}
# [idx_to_artist[i[0]] for i in similar_movie]

In [55]:
# 위의 코드를 이용하여 유사한 영화 추천해주는 함수 만들기
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie


In [56]:
get_similar_movie('Aladdin (1992)')

['Aladdin (1992)',
 'Beauty and the Beast (1991)',
 'Lion King, The (1994)',
 'Little Mermaid, The (1989)',
 'Hunchback of Notre Dame, The (1996)',
 'Mulan (1998)',
 'Hercules (1997)',
 'Toy Story (1995)',
 'Tarzan (1999)',
 'Anastasia (1997)']

#### 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [59]:
user = user_to_idx['mee']
# recommend에서는 user*item CSR Matrix를 받습니다.
model_recommended = als_model.recommend(user, csr_ratings, N=20, filter_already_liked_items=True)
model_recommended

[(50, 0.40874344),
 (322, 0.30587175),
 (4, 0.23013586),
 (458, 0.22399762),
 (545, 0.22099152),
 (476, 0.19817272),
 (33, 0.19381317),
 (1130, 0.18751329),
 (60, 0.1862943),
 (29, 0.1753729),
 (330, 0.16735452),
 (480, 0.15710624),
 (596, 0.14461833),
 (10, 0.14037628),
 (478, 0.13824394),
 (245, 0.13664372),
 (474, 0.13623384),
 (678, 0.1337342),
 (32, 0.13032205),
 (173, 0.12926477)]

In [61]:
[idx_to_movie[i[0]] for i in model_recommended]

['Toy Story 2 (1999)',
 'Babe (1995)',
 "Bug's Life, A (1998)",
 'Mask, The (1994)',
 'Santa Clause, The (1994)',
 'Wrong Trousers, The (1993)',
 'Aladdin (1992)',
 'Indian in the Cupboard, The (1995)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Close Shave, A (1995)',
 'Lion King, The (1994)',
 'Grand Day Out, A (1992)',
 'Hook (1991)',
 'Beauty and the Beast (1991)',
 "Wayne's World (1992)",
 'Player, The (1992)',
 'Chicken Run (2000)',
 'Willy Wonka and the Chocolate Factory (1971)',
 'Hercules (1997)',
 'Dragonheart (1996)']

In [68]:
# 왜 추천해줬을까? 기여도 분석 - explain 메소드 사용
Beauty_beast = movie_to_idx['Beauty and the Beast (1991)']
explain = als_model.explain(user, csr_ratings, itemid=Beauty_beast)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Toy Story (1995)', 0.12662663989941958),
 ('Jumanji (1995)', 0.013481916484545924),
 ('One Little Indian (1973)', 0.0015081169312384617),
 ("'One Flew Over the Cuckoo's Nest (1975)", 0.00043071510990253957),
 ('To Die For (1995)', -0.004015026407722514)]

---
## 루브릭 평가 요구사항  
  
### 1) CSR matrix가 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들어졌다.
-  영화 갯수 : 3628 , 사용자 수 : 6039 에 선호하는 데이터를 더 추가하여 CSR matrix를 통해 사용자수는 6040으로 늘어난 것을 볼 수 있다.
  
### 2) MF 모델이 정상적으로 훈련되어 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
-  MF 모델 훈련을 통해 사용자 벡터와 비교할 벡터 내적값으로 유사도를 확인하였다.  

### 3)   비슷한 영화 찾기와 유저에게 추천하기의 과정에서 MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.
-  similar_items 메서드, recommend 메서드, explain 메소드를 이용하여 영화 추천과 기여도를 확인하였다.
---

# 회고  
---- 
- 1) ratings와 movies 병합 과정이 없으니까 아래 코드들에서 movies에 title을 찾으려 할때 오류가 났다. 
    - --> 두가지에서 가져오지 않고, 하나로 병합 후 ratings_add에서 title을 찾으니 해결!  
---
- 2) 내가 선호하는 영화 5가지를 추가할 때, 새로운 영화를 입력하는 줄 알았다. 이때 movie_id가 없어서 계속 오류 발생 -> user_id 인덱싱/정수화가 안된다. (무슨 연관인지는 모르겠음)
    - --> 데이터셋 내에서 선호하는 영화 5가지를 넣어줬다. -> 이때 각각의 movie_id 찾고 싶었다.
        - --> 수동으로 영화의 Id를 찾아줬으나, 자동으로 찾아주는 함수를 만들고 싶었다. 이후 공부해볼 계획이다!  

- 3)  인덱스 차원 오류 나타나서 빼줌 - shape= (num_user, num_movie)  
----  
오류가 나고 수정을 하면 이전 실행 값을 저장하고 있어서 계속 restart를 해야했다. 코드양이 적어서 다행이였다.  
다른 영화로 실행했을 때 두 벡터를 내적한 값이 너무 작거나 천차만별이였으며, 가끔은 음수로도 나타났다. 데이터양이 적어서 그런지 훈련을 잘못 시킨것 인지 좀 더 알아봐야겠다.   
이전 프로젝트보다 더 한줄 한줄 이해하려고 노력했고, 변수명도 바꾸면서 필요한 부분은 더 추가하여 만들었다. 그래도 완벽히 숙지한 것도 아니고 궁금한 부분들도 많지만 스스로에게 조금은 뿌듯한 프로젝트였다 :)